In [1]:
!python --version
import tensorflow as tf
print(tf.__version__)

Python 3.7.9


c:\users\14106\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\14106\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\14106\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\14106\appd

1.13.1


In [1]:
YEAR='2022'
SCHOOL_PATH="C:\\Users\\14106\\GAT\\100school\\"
SOURCE_PATH="C:\\Users\\14106\\GAT\\preprocessed"+YEAR+"\\"
DEST_PATH='C:\\Users\\14106\\GAT\\result\\'
CODE_PATH="C:\\Users\\14106\\GAT\\"
ORIG_UNIV=['columbia',"notredame","uofm","UCSD",'berkeley',"Harvard","ucla", 'dartmouth']
ADJ='CCasym'

import importlib
import time
import tqdm
import numpy as np
import os
os.chdir(CODE_PATH)
from gat import GAT, HeteGAT, HeteGAT_multi
import process
import importlib
from layers import attn_head, SimpleAttLayer
import pandas as pd
import pickle
import scipy.sparse
from scipy.sparse import csc_matrix
from scipy import sparse
import scipy.io as sio
import scipy.sparse as sp
import csv 
import tensorflow as tf

def get_school_name():
    files = os.listdir(SOURCE_PATH)
    files = [f[16:-4] for f in files if f[0:15]=='feature_sampled']
    return files

def get_subreddits():
    os.chdir(SCHOOL_PATH)
    dat = pd.read_csv('colleges-CCIHE-final.csv')
    schools = []
    for i in range(len(dat.name)):
        schools.append(dat.loc[i, 'subreddit'])
    return schools

def sample_mask(idx, l):
    """Create mask."""
    mask = np.zeros(l)
    mask[idx] = 1
    return np.array(mask, dtype=np.bool)

def load_data_dblp(s,path=SOURCE_PATH):
    #schools = ["notredame","uofm","columbia","dartmouth","UCSD","berkeley","Harvard","ucla"]
    #s=schools[3]#use this index to control which network we are going to run.
    t="comment"
    
    os.chdir(path)
    name="label_emt3_"+s+".csv"
    #truelabels=pd.read_csv(name,skip_blank_lines=True,header=None)
    name="feature_sampled_"+s+".csv"
    truefeatures=pd.read_csv(name,skip_blank_lines=True,header=None).values
    #name="feature_"+s+t+".npz"
    #truefeatures=scipy.sparse.load_npz(name)
    #truefeatures=truefeatures.toarray()

    N=truefeatures.shape[0]
    name=ADJ+"_sampled_"+s+".npz"
    #name="CCasym_"+s+".npz"
    dat_cc=scipy.sparse.load_npz(name);
    dat_cc.transpose(axes=None, copy=False)
    dat_cc=dat_cc.toarray();
    rownetworks = [(dat_cc)]

    y=np.zeros((N,2))
    name="train_index_"+s+"_cm.p"
    train_idx=np.array([0,1])
    val_idx=np.array([2,3])
    test_idx=np.array(range(N))
    print(N)
    
    train_mask = sample_mask(train_idx, y.shape[0])
    val_mask = sample_mask(val_idx, y.shape[0])
    test_mask = sample_mask(test_idx, y.shape[0])# ?? why copy three times? First for center node, 2 for each metapath.
    
    y_train = np.zeros(y.shape)
    y_val = np.zeros(y.shape)
    y_test = np.zeros(y.shape)
    #y=y.values
    #y_train[train_mask, :] = y[train_mask, :]
    #y_val[val_mask, :] = y[val_mask, :]
    #y_test[test_mask, :] = y[test_mask, :]

    # return selected_idx, selected_idx_2
    print('y_train:{}, y_val:{}, y_test:{}, train_idx:{}, val_idx:{}, test_idx:{}'.format(y_train.shape,
                                                                                          y_val.shape,
                                                                                          y_test.shape,
                                                                                          train_idx.shape,
                                                                                          val_idx.shape,
                                                                                          test_idx.shape))
    truefeatures_list = [truefeatures, truefeatures]#, truefeatures, truefeatures]# ?? why copy three times? First for center node, 2 for each metapath.    
    os.chdir(CODE_PATH)
    return rownetworks, truefeatures_list, y_train, y_val, y_test, train_mask, val_mask, test_mask



c:\users\14106\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\14106\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\14106\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\14106\appd

In [2]:
#iterate all testing dataset to get prediction of all set
#the model used and type of classification depends on paths from previous block
#don't run this without running the previous one, unless you specify path again in this script
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3"

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

dataset = 'acm'
featype = 'fea'
# training params

checkpt_file=DEST_PATH+'acm_allMP_multi_fea_.ckpt'
#checkpt_file = 'C:\data\HAN-modified\\tranfer learning\\acm_allMP_multi_fea_2cls.ckpt'

print('model: {}'.format(checkpt_file))
# training params for only CC
batch_size = 1
nb_epochs = 100
patience = 100
lr = 0.02  # learning rate
l2_coef = 0.15#0.1#0.001  # weight decay
# numbers of hidden units per each attention head in each layer
hid_units = [8]
n_heads = [4, 1]  # additional entry for the output layer
residual = False
nonlinearity = tf.nn.elu
model = HeteGAT_multi

# jhy data


#path_cur='C:\\data\\HAN-modified\\tranfer learning\\'
#SOURCE_PATH='C:\\data\\HAN-modified\\data\\preprocessed-trial\\'

schools = get_subreddits()
#schools =["notredame2019","notredame2020","uofm2019","uofm2020","columbia2019","columbia2020","dartmouth","UCSD2019","UCSD2020","berkeley2019","berkeley2020","Harvard2019","Harvard2020","ucla2019","ucla2020"]

#SOURCE_PATH='C:\\data\\HAN-modified\\tranfer learning\\'
i=0
   

model: C:\Users\14106\GAT\result\acm_allMP_multi_fea_.ckpt


In [5]:
schools[65]

'williamandmary'

In [6]:
#schools=ORIG_UNIV
#schools=['uofm']

for i in tqdm.tqdm(range(66,len(schools),1)):
    adj_list, fea_list, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data_dblp(s=schools[i])
    

    if featype == 'adj':
        fea_list = adj_list

    nb_nodes = fea_list[0].shape[0]
    ft_size = fea_list[0].shape[1]
    nb_classes = y_train.shape[1]

    # adj = adj.todense()
    
    # features = features[np.newaxis]  # [1, nb_node, ft_size]
    print(['dim',len(fea_list),len(fea_list[0]),len(fea_list[0][0])])
    fea_list = [fea[np.newaxis] for fea in fea_list]
    print(['dim',len(fea_list),len(fea_list[0]),len(fea_list[0][0])])
    adj_list = [adj[np.newaxis] for adj in adj_list]
    y_train = y_train[np.newaxis]
    y_val = y_val[np.newaxis]
    y_test = y_test[np.newaxis]
    train_mask = train_mask[np.newaxis]
    val_mask = val_mask[np.newaxis]
    test_mask = test_mask[np.newaxis]
    print("s")
    #biases_list = [process.adj_to_bias(adj, [nb_nodes], nhood=1) for adj in adj_list
    #biases_list = [process.adj_to_bias(adj.astype('float') ) for adj in adj_list]
    
    biases_list = [process.adj_to_bias(adj, [nb_nodes], nhood=1) for adj in adj_list]
    import pickle
    name=schools[i]+YEAR+'adj.p'
    os.chdir(SOURCE_PATH)
    #pickle.dump(biases_list,open(name,"wb"))
    #biases_list=pickle.load(open(name,"rb"))
    
    #For two year separate
    print('build graph...')
    #checkpt_file = 'C:\data\HAN-modified\\tranfer learning\model.ckpt'
    with tf.Graph().as_default():
        with tf.name_scope('input'):
            ftr_in_list = [tf.placeholder(dtype=tf.float32,
                                          shape=(batch_size, nb_nodes, ft_size),
                                          name='ftr_in_{}'.format(i))
                           for i in range(len(fea_list))]
            bias_in_list = [tf.placeholder(dtype=tf.float32,
                                           shape=(batch_size, nb_nodes, nb_nodes),
                                           name='bias_in_{}'.format(i))
                            for i in range(len(biases_list))]
            lbl_in = tf.placeholder(dtype=tf.int32, shape=(
                batch_size, nb_nodes, nb_classes), name='lbl_in')
            msk_in = tf.placeholder(dtype=tf.int32, shape=(batch_size, nb_nodes),
                                    name='msk_in')
            attn_drop = tf.placeholder(dtype=tf.float32, shape=(), name='attn_drop')
            ffd_drop = tf.placeholder(dtype=tf.float32, shape=(), name='ffd_drop')
            is_train = tf.placeholder(dtype=tf.bool, shape=(), name='is_train')
        # forward
        logits, final_embedding, att_val = model.inference(ftr_in_list, nb_classes, nb_nodes, is_train,attn_drop, ffd_drop,bias_mat_list=bias_in_list,hid_units=hid_units, n_heads=n_heads,residual=residual, activation=nonlinearity)
    
        # cal masked_loss
        log_resh = tf.reshape(logits, [-1, nb_classes])
        lab_resh = tf.reshape(lbl_in, [-1, nb_classes])
        msk_resh = tf.reshape(msk_in, [-1])
        loss = model.masked_softmax_cross_entropy(log_resh, lab_resh, msk_resh)
        accuracy = model.masked_accuracy(log_resh, lab_resh, msk_resh)
        # optimzie
        train_op = model.training(loss, lr, l2_coef)
        vlss_mn = np.inf
        vacc_mx = 0.0
        curr_step = 0
        saver = tf.train.Saver()
        init_op = tf.group(tf.global_variables_initializer(),
                           tf.local_variables_initializer())
        #new_saver = tf.train.import_meta_graph('C:\data\HAN-modified\\tranfer learning\model.ckpt.meta')
    
    
        with tf.Session(config=config) as sess:
            #sess.run(init_op)

            train_loss_avg = 0
            train_acc_avg = 0
            val_loss_avg = 0
            val_acc_avg = 0
            os.chdir(CODE_PATH)
            saver.restore(sess, DEST_PATH+"acm_allMP_multi_fea_.ckpt")
            #saver.restore(sess, "C:\data\HAN-modified\\tranfer learning\\model.ckpt")
            #new_saver.restore(sess, tf.train.latest_checkpoint('./'))
            print("Model restored.")
            
            #saver.restore(sess, checkpt_file)
            #print('load model from : {}'.format(checkpt_file))
            ts_size = fea_list[0].shape[0]
            print(fea_list[0].shape)
            ts_step = 0
            ts_loss = 0.0
            ts_acc = 0.0
            print("test start")
            while ts_step * batch_size < ts_size:
                print(ts_step)
                print(batch_size)
                print(ts_size)
                # fd1 = {ftr_in: features[ts_step * batch_size:(ts_step + 1) * batch_size]}
                fd1 = {i: d[ts_step * batch_size:(ts_step + 1) * batch_size]
                       for i, d in zip(ftr_in_list, fea_list)}
                fd2 = {i: d[ts_step * batch_size:(ts_step + 1) * batch_size]
                       for i, d in zip(bias_in_list, biases_list)}
                fd3 = {lbl_in: y_test[ts_step * batch_size:(ts_step + 1) * batch_size],
                       msk_in: test_mask[ts_step * batch_size:(ts_step + 1) * batch_size],
                
                       is_train: False,
                       attn_drop: 0.0,
                       ffd_drop: 0.0}
            
                fd = fd1
                fd.update(fd2)
                fd.update(fd3)
                
                loss_value_ts, acc_ts, jhy_final_embedding, predicted = sess.run([loss, accuracy, final_embedding, log_resh],
                                                                      feed_dict=fd)
                import pickle
                name=schools[i]+'_'+YEAR+'.p'
                os.chdir(DEST_PATH)
                pickle.dump(predicted,open(name,"wb"))
                ts_loss += loss_value_ts
                ts_acc += acc_ts
                ts_step += 1

            print('Test loss:', ts_loss / ts_step,
                  '; Test accuracy:', ts_acc / ts_step)
    
            
            sess.close()


  0%|                                                                                | 0/62 [00:00<?, ?it/s]

19024
y_train:(19024, 2), y_val:(19024, 2), y_test:(19024, 2), train_idx:(2,), val_idx:(2,), test_idx:(19024,)
['dim', 2, 19024, 768]
['dim', 2, 1, 19024]
s
build graph...
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.conv1d instead.
Instructions for updating:
Colocations handled automatically by placer.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.dense instead.
de
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use standard file

  2%|█                                                                  | 1/62 [19:35<19:55:26, 1175.84s/it]

Test loss: 0.0 ; Test accuracy: 0.4359756112098694
28416
y_train:(28416, 2), y_val:(28416, 2), y_test:(28416, 2), train_idx:(2,), val_idx:(2,), test_idx:(28416,)
['dim', 2, 28416, 768]
['dim', 2, 1, 28416]
s
build graph...
de
INFO:tensorflow:Restoring parameters from C:\Users\14106\GAT\result\acm_allMP_multi_fea_.ckpt
Model restored.
(1, 28416, 768)
test start
0
1
1


  3%|██                                                               | 2/62 [1:14:59<40:39:05, 2439.09s/it]

...

Test loss: 0.0 ; Test accuracy: 0.257400244474411
19215
y_train:(19215, 2), y_val:(19215, 2), y_test:(19215, 2), train_idx:(2,), val_idx:(2,), test_idx:(19215,)
['dim', 2, 19215, 768]
['dim', 2, 1, 19215]
s
build graph...
de
INFO:tensorflow:Restoring parameters from C:\Users\14106\GAT\result\acm_allMP_multi_fea_.ckpt
Model restored.
(1, 19215, 768)
test start
0
1
1


 98%|█████████████████████████████████████████████████████████████████▉ | 61/62 [11:05:48<06:42, 402.79s/it]

Test loss: 0.0 ; Test accuracy: 0.4175904095172882
8474
y_train:(8474, 2), y_val:(8474, 2), y_test:(8474, 2), train_idx:(2,), val_idx:(2,), test_idx:(8474,)
['dim', 2, 8474, 768]
['dim', 2, 1, 8474]
s
build graph...
de
INFO:tensorflow:Restoring parameters from C:\Users\14106\GAT\result\acm_allMP_multi_fea_.ckpt
Model restored.
(1, 8474, 768)
test start
0
1
1


100%|███████████████████████████████████████████████████████████████████| 62/62 [11:08:43<00:00, 647.15s/it]

Test loss: 0.0 ; Test accuracy: 0.3627566695213318


# New Section

In [49]:
os.chdir('C:\\Users\\14106\\GAT\\result\\')
#ORIG_UNIV=['columbia',"notredame","uofm","UCSD",'berkeley',"Harvard","ucla", 'dartmouth']`
s=ORIG_UNIV[2]
name=s+'_'+YEAR+'.p'
predicted1=pickle.load(open(name,'rb'))
print(predicted1)
len(predicted1)

[[ 0.79152066 -0.9221296 ]
 [-1.1336458   1.2642779 ]
 [ 0.8228782  -0.95387465]
 ...
 [-0.66938376  0.74347806]
 [-0.6311226   0.6883371 ]
 [-1.1250875   1.2593728 ]]


12524

In [50]:
os.chdir('C:\\Users\\14106\\Downloads\\re1\\')
name=s+YEAR+'.p'
predicted=pickle.load(open(name,'rb'))
print(predicted)
len(predicted)

[[ 0.79152066 -0.9221295 ]
 [-1.1336457   1.2642778 ]
 [ 0.8228783  -0.95387477]
 ...
 [-0.66938365  0.7434779 ]
 [-0.6311226   0.688337  ]
 [-1.1250874   1.2593727 ]]


12524

In [31]:
#biases_list1=biases_list#[0][0][0][1]
list((biases_list1[0][0][0])!=(biases_list[0][0][0])).index(1)

9702

In [67]:
os.chdir("C:\\Users\\14106\\GAT\\preprocessed2020\\")
#ORIG_UNIV=['columbia',"notredame","uofm","UCSD",'berkeley',"Harvard","ucla", 'dartmouth']`
s=ORIG_UNIV[0]
name="CCsym_sampled_"+s+".npz"
dat_cc1=scipy.sparse.load_npz(name);
name="CCasym_sampled_"+s+".npz"
dat_cc=scipy.sparse.load_npz(name);
print(dat_cc.sum())
print(dat_cc1.sum())

3488
3488


3488
6976


In [43]:
adj_list[0][0][0][9702]
adj_list[0][0][9702][0]

1